# Compose: Training a model to generate music

In [1]:
import os
import pickle
import numpy
import glob

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model
import tensorflow.keras

from mido import MidiFile ,MetaMessage, Message, MidiTrack, tick2second

from models.LSTM_Minimized import get_distinct, create_lookups, prepare_sequences, create_network

Using TensorFlow backend.


## Set parameters

In [2]:
# run params
section = 'compose'
run_id = '1122'
music_name = 'midis'

run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])


store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join('data', music_name)

if not os.path.exists(run_folder):
    ### EDITED #################
    # Failes if subdirectorys (1120/midis) missing 
    # os.mkdir(run_folder)
    # Creates all needed subdirectorys (1120/midis) 
    os.makedirs(run_folder)
    ############################
    os.mkdir(os.path.join(run_folder, 'store'))
    os.mkdir(os.path.join(run_folder, 'output'))
    os.mkdir(os.path.join(run_folder, 'weights'))
    os.mkdir(os.path.join(run_folder, 'viz'))
    


mode = 'build' # 'load' # 

# data params
#intervals = range(1)
seq_len = 64

# model params
embed_size = 200
rnn_units = 256
use_attention = True

## Extract the notes

In [3]:
# Returns a list of all the midi Files in the given data_folder and its subfolders
def get_Music_List(directory):
    global music_list
    music_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".mid"):
                music_list.append(os.path.join(root, file))
                #print(os.path.join(root, file))               
                #(Quelle: https://stackoverflow.com/questions/3964681/find-all-files-in-a-directory-with-extension-txt-in-python)
    return music_list

In [4]:
music_list = get_Music_List(data_folder)
len(music_list)

127

In [5]:
isOnlyMonoTrackMode = True
monoTrackMidis = []

if mode == 'build':
    
    music_list = get_Music_List(data_folder)
    print(len(music_list), 'files in total')

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        midi_score = MidiFile(os.path.join(file))
        if (midi_score.type != 0):
            #print("Skipped: %s", midi_score)
            continue
        print(midi_score)
        monoTrackMidis.append(midi_score)
    print(len(monoTrackMidis))

127 files in total
1 Parsing ../data\midis\73561_08.mid
<midi file '../data\\midis\\73561_08.mid' type 0, 1 tracks, 17714 messages>
2 Parsing ../data\midis\Backstreet Boys - I Want It That Way.mid
3 Parsing ../data\midis\Base Attack - Nobody Listen To Techno (Ritmika Dance 2009 remix).mid
4 Parsing ../data\midis\Bohemian-Rhapsody-1.mid
5 Parsing ../data\midis\crazylittlethingscalledlove.mid
6 Parsing ../data\midis\Guns n Roses - Sweet Child O Mine.mid
7 Parsing ../data\midis\Headhunterz - Scrap Attack (defqon.1 2009 Anthem)(Endymion Remix.mid
8 Parsing ../data\midis\John Denver - Take Me Home Country Roads.mid
9 Parsing ../data\midis\Kaoma - Lambada 3.mid
10 Parsing ../data\midis\Kayem vs. El Grekoz - Dance Like A Pro (Da Tweekaz 'NES' Mix).mid
11 Parsing ../data\midis\Mario Bros. - Super Mario Bros. Theme.mid
12 Parsing ../data\midis\Michael Jackson - Beat It.mid
13 Parsing ../data\midis\Michael Jackson - Billie Jean.mid
14 Parsing ../data\midis\music59.mid
<midi file '../data\\midis\

79 Parsing ../data\midis\Lady Gaga\Monster.mid
80 Parsing ../data\midis\Lady Gaga\NothinElseICanSay.mid
81 Parsing ../data\midis\Lady Gaga\Paparazzi.mid
<midi file '../data\\midis\\Lady Gaga\\Paparazzi.mid' type 0, 1 tracks, 1983 messages>
82 Parsing ../data\midis\Lady Gaga\PokerFace.mid
83 Parsing ../data\midis\Lady Gaga\TheEdgeOfGlory.mid
<midi file '../data\\midis\\Lady Gaga\\TheEdgeOfGlory.mid' type 0, 1 tracks, 2555 messages>
84 Parsing ../data\midis\piano_only\Avicii - Levels (Skrillex remix).mid
85 Parsing ../data\midis\piano_only\Biogra11.mid
86 Parsing ../data\midis\piano_only\D-Sturb-Sefa-Nothing-Like-The-Old-School-frozensunrise-20190127172807-nonstop2k.com.mid
87 Parsing ../data\midis\piano_only\D-Sturb-Sefa-Nothing-Like-The-Oldschool-Intro-Anonymous-20200205123836-nonstop2k.com.mid
88 Parsing ../data\midis\piano_only\Da Tweekaz - Examination Of Time.mid
89 Parsing ../data\midis\piano_only\Da Tweekaz - Healing Incantation.mid
90 Parsing ../data\midis\piano_only\Da Tweekaz -

# Preprocessing Data for Tokenization

In [6]:
commands = []
values1 = []
durations = []

for midi in monoTrackMidis:
    
    print('\n' + str(midi))
    #print(midi.ticks_per_beat)
    
    commands.append('START')
    values1.append(-1)
    durations.append(0)
    
    # Set tempo to default until tempo message was read
    pTempo = 500000
    
    # All midis in the List only have one Track so we just need to look at that one
    for i, msg in enumerate(midi.tracks[0]):
        if (msg.time == 0):
            time = 0
        else:
            # Convert the time into seconds and roun to 0.1 place
            time = round(tick2second(msg.time, midi.ticks_per_beat, pTempo), 1)
        if type(msg) == MetaMessage:
            if msg.type == 'set_tempo':
                pTempo = msg.tempo
                #print(pTempo)
                continue
        elif type(msg) == Message:
            if(msg.type == 'sysex' or msg.type == 'pitchwheel' or msg.type == 'control_change' or msg.type == 'program_change'):
                continue
                
            commands.append(msg.type + '.' + str(msg.channel))
            durations.append(time)
            
            if msg.type == 'note_on' or msg.type == 'note_off':
                values1.append(msg.note)
            elif msg.type == 'program_change':
                values1.append(msg.program)
            else:
                print(msg)



<midi file '../data\\midis\\73561_08.mid' type 0, 1 tracks, 17714 messages>

<midi file '../data\\midis\\music59.mid' type 0, 1 tracks, 7943 messages>

<midi file '../data\\midis\\R.ASTLEY.Never gonna give you up K.mid' type 0, 1 tracks, 16360 messages>

<midi file '../data\\midis\\ABBA\\AngelEyes.mid' type 0, 1 tracks, 4003 messages>

<midi file '../data\\midis\\ABBA\\Chiquitita.mid' type 0, 1 tracks, 539 messages>

<midi file '../data\\midis\\ABBA\\HoneyHoney.mid' type 0, 1 tracks, 3026 messages>

<midi file '../data\\midis\\ABBA\\IDoIDoIDo.mid' type 0, 1 tracks, 1894 messages>

<midi file '../data\\midis\\ABBA\\MammaMia.mid' type 0, 1 tracks, 2538 messages>

<midi file '../data\\midis\\ABBA\\MoneyMoneyMoney.mid' type 0, 1 tracks, 2114 messages>

<midi file '../data\\midis\\ABBA\\MyLoveMyLife.mid' type 0, 1 tracks, 499 messages>

<midi file '../data\\midis\\ABBA\\SoLong(MayRemove).mid' type 0, 1 tracks, 2148 messages>

<midi file '../data\\midis\\ABBA\\SummerNightCity.mid' type 0, 1

## Create the lookup tables

In [7]:
# get the distinct sets of all input data
command_names, n_commands = get_distinct(commands)
value1_names, n_value1 = get_distinct(list(map(str, values1)))
duration_names, n_durations = get_distinct(durations)

distincts = [command_names, n_commands, value1_names, n_value1, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

# make the lookup dictionaries for notes and dictionaries and save
command_to_int, int_to_command = create_lookups(command_names)
value1_to_int, int_to_value1 = create_lookups(value1_names)
duration_to_int, int_to_duration = create_lookups(duration_names)

lookups = [command_to_int, int_to_command, value1_to_int, int_to_value1, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

In [8]:
#print('\nvalue2_to_int')
command_to_int

{'START': 0,
 'note_off.0': 1,
 'note_off.1': 2,
 'note_off.10': 3,
 'note_off.11': 4,
 'note_off.12': 5,
 'note_off.13': 6,
 'note_off.14': 7,
 'note_off.15': 8,
 'note_off.2': 9,
 'note_off.3': 10,
 'note_off.4': 11,
 'note_off.5': 12,
 'note_off.6': 13,
 'note_off.7': 14,
 'note_off.8': 15,
 'note_off.9': 16,
 'note_on.0': 17,
 'note_on.1': 18,
 'note_on.10': 19,
 'note_on.11': 20,
 'note_on.12': 21,
 'note_on.13': 22,
 'note_on.14': 23,
 'note_on.15': 24,
 'note_on.2': 25,
 'note_on.3': 26,
 'note_on.4': 27,
 'note_on.5': 28,
 'note_on.6': 29,
 'note_on.7': 30,
 'note_on.8': 31,
 'note_on.9': 32}

In [9]:
#print('\nduration_to_int')
duration_to_int

{0: 0,
 0.1: 1,
 0.2: 2,
 0.3: 3,
 0.4: 4,
 0.5: 5,
 0.6: 6,
 0.7: 7,
 0.8: 8,
 0.9: 9,
 1.0: 10,
 1.1: 11,
 1.2: 12,
 1.3: 13,
 1.4: 14,
 1.5: 15,
 1.6: 16,
 2.8: 17,
 5.8: 18,
 6.9: 19}

## Prepare the sequences used by the Neural Network

In [10]:
values1_strings = list(map(str, values1))
network_input, network_output = prepare_sequences(commands, values1_strings, durations, lookups, distincts, seq_len)

In [11]:
print('#####  INPUT  ####')
print('command input')
print(network_input[0][0])
print('values1 input')
print(network_input[1][0])
print('durations input')
print(network_input[2][0])

print('\n#####  OUTPUT  ####')
print('command output')
print(network_output[0][0])
print('values1 output')
print(network_output[0][1])
print('durations output')
print(network_output[0][2])

#####  INPUT  ####
command input
[ 0 25 25 25 25  9  9  9 25 25  9 25 25  9 25  9  9  9  9 25 25 25 25  9
 25  9  9  9  9 30 14 25 25 25 25  9 25  9  9  9  9 25 25 25 25  9  9 25
 25 30  9 14 25  9 25  9  9  9 25 25 20 25  9  9]
values1 input
[ 0 43 46 50 55 46 43 55 42 57 50 50 46 57 57 46 50 42 57 41 58 50 46 58
 58 58 46 50 41 46 46 40 52 48 43 52 52 43 52 48 40 39 50 46 43 46 50 46
 50 43 46 43 46 43 43 43 46 39 33 41 41 45 45 33]
durations input
[0 0 0 4 4 1 0 1 0 0 2 2 4 2 2 2 0 0 0 2 0 4 4 2 2 2 0 0 0 0 1 1 0 4 4 2 1
 2 0 0 0 0 0 4 4 3 1 1 8 2 1 0 1 3 1 1 1 1 0 0 0 0 3 0]

#####  OUTPUT  ####
command output
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
values1 output
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
durations output
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Create the structure of the neural network

In [12]:
model, att_model = create_network(n_commands, n_value1, n_durations, embed_size, rnn_units, use_attention)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
commands_channels_in (InputLaye [(None, None)]       0                                            
__________________________________________________________________________________________________
values1_in (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
durations_in (InputLayer)       [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    6600        commands_channels_in[0][0]       
______________________________________________________________________________________________

In [13]:
#plot_model(model, to_file=os.path.join(run_folder ,'viz/model_simplified_max2.png'), show_shapes = True, show_layer_names = True)

## Train the neural network

In [14]:
# Used to continue Training on given weights
weights_folder = os.path.join(run_folder, 'weights')
#model.load_weights(os.path.join(weights_folder, "weights_custom.h5"))

In [ ]:
#weights_folder = os.path.join(run_folder, "weights")

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger_custom.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights_custom.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    #, early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights_custom.h5"))
model.fit(network_input, network_output
          , epochs=2000000, batch_size=128
          , validation_split = 0.1
          , callbacks=callbacks_list
          , shuffle=True)



Train on 114111 samples, validate on 12679 samples
Epoch 1/2000000
114111/114111 [==============================] - 34s 296us/sample - loss: 6.0979 - commands_channels_out_loss: 2.5268 - values1_out_loss: 3.1630 - durations_out_loss: 0.4076 - val_loss: 6.5280 - val_commands_channels_out_loss: 2.4692 - val_values1_out_loss: 3.4991 - val_durations_out_loss: 0.5873
Epoch 2/2000000
114111/114111 [==============================] - 30s 260us/sample - loss: 5.4289 - commands_channels_out_loss: 2.2803 - values1_out_loss: 2.7669 - durations_out_loss: 0.3815 - val_loss: 6.7599 - val_commands_channels_out_loss: 2.5354 - val_values1_out_loss: 3.6377 - val_durations_out_loss: 0.6225
Epoch 3/2000000
114111/114111 [==============================] - 30s 260us/sample - loss: 5.2320 - commands_channels_out_loss: 2.1891 - values1_out_loss: 2.6688 - durations_out_loss: 0.3743 - val_loss: 6.8434 - val_commands_channels_out_loss: 2.6697 - val_values1_out_loss: 3.5926 - val_durations_out_loss: 0.6136
Epoch 4

In [ ]:
model.save_weights(os.path.join(weights_folder, "weights_finished.h5"))